In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

import warnings
warnings.filterwarnings("ignore")

# Global visual style upgrade
sns.set_theme(style="whitegrid", palette="deep")
plt.rcParams["figure.figsize"] = (10, 6)
plt.rcParams["axes.titlesize"] = 14
plt.rcParams["axes.labelsize"] = 12


In [ ]:
df = pd.read_csv("dataset.csv")
df.shape, df.head()


In [ ]:
# Check missing values
df.isnull().sum()

# Drop duplicates
df = df.drop_duplicates()

# Reset index
df = df.reset_index(drop=True)

df.shape


In [ ]:
# Keep only final outcomes
df = df[df["Target"].isin(["Dropout", "Graduate"])]

# Convert to binary
df["pass"] = df["Target"].map({
    "Dropout": 0,
    "Graduate": 1
})

# Drop original target column
df = df.drop(columns=["Target"])

df["pass"].value_counts(normalize=True), df.shape


In [ ]:
sns.countplot(x="pass", data=df)
plt.xticks([0,1],["Dropout (Fail)","Graduate (Pass)"])
plt.title("Final Student Outcome Distribution")
plt.show()


In [ ]:
plt.figure(figsize=(12,8))
corr = df.corr()
sns.heatmap(corr, cmap="coolwarm")
plt.title("Feature Correlation Heatmap")
plt.show()


In [ ]:
X = df.drop(columns=["pass"])
y = df["pass"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [ ]:
models = {
    "Logistic Regression": LogisticRegression(class_weight={0:2,1:1}),
    "KNN": KNeighborsClassifier(),
    "Decision Tree": DecisionTreeClassifier(class_weight="balanced"),
    "Random Forest": RandomForestClassifier(class_weight="balanced", n_estimators=300, random_state=42)
}

results = []

for name, model in models.items():
    if name in ["Decision Tree", "Random Forest"]:
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
    else:
        model.fit(X_train_scaled, y_train)
        y_pred = model.predict(X_test_scaled)

    results.append({
        "Model": name,
        "Accuracy": accuracy_score(y_test, y_pred),
        "Precision": precision_score(y_test, y_pred),
        "Recall": recall_score(y_test, y_pred),
        "F1": f1_score(y_test, y_pred)
    })

results_df = pd.DataFrame(results)
results_df


In [ ]:
sns.barplot(x="Model", y="F1", data=results_df)
plt.title("Model Comparison (F1-Score)")
plt.show()


In [ ]:
best_model_name = results_df.sort_values(by="F1", ascending=False).iloc[0]["Model"]
best_model = models[best_model_name]

if best_model_name in ["Decision Tree", "Random Forest"]:
    best_model.fit(X_train, y_train)
    y_pred_best = best_model.predict(X_test)
else:
    best_model.fit(X_train_scaled, y_train)
    y_pred_best = best_model.predict(X_test_scaled)

print(classification_report(y_test, y_pred_best))


In [ ]:
cm = confusion_matrix(y_test, y_pred_best)
sns.heatmap(cm, annot=True, fmt="d")
plt.title("Confusion Matrix")
plt.show()


In [ ]:
if best_model_name in ["Decision Tree", "Random Forest"]:
    importances = best_model.feature_importances_
else:
    rf = RandomForestClassifier(class_weight="balanced", n_estimators=300, random_state=42)
    rf.fit(X_train, y_train)
    importances = rf.feature_importances_

importance_df = pd.DataFrame({
    "Feature": X.columns,
    "Importance": importances
}).sort_values(by="Importance", ascending=False).head(12)

sns.barplot(x="Importance", y="Feature", data=importance_df)
plt.title("Top 12 Most Important Features")
plt.show()


In [ ]:
def predict_student_outcome(**inputs):
    sample = pd.DataFrame([inputs])

    # Align features
    sample = sample.reindex(columns=X.columns, fill_value=0)

    if best_model_name in ["Decision Tree", "Random Forest"]:
        sample_final = sample
    else:
        sample_final = scaler.transform(sample)

    pred = best_model.predict(sample_final)[0]

    if hasattr(best_model, "predict_proba"):
        prob = best_model.predict_proba(sample_final)[0][pred]
    else:
        prob = None

    result = "GRADUATE ✅" if pred == 1 else "DROPOUT ❌"

    print("━━━━━━━━━━━━━━━━━━━━━━━━")
    print("STUDENT RETENTION AI")
    print("━━━━━━━━━━━━━━━━━━━━━━━━")
    print("Prediction:", result)
    if prob is not None:
        print("Confidence:", f"{prob*100:.2f}%")
    print("Model:", best_model_name)
    print("━━━━━━━━━━━━━━━━━━━━━━━━")

    return pred, prob


In [ ]:
def generate_random_student(show=True):
    """
    Generates a random student profile from the dataset
    and optionally prints their attributes.
    """
    sample = df.sample(1).drop(columns=["pass"])
    student_dict = sample.iloc[0].to_dict()

    if show:
        print("\n🎲 RANDOM STUDENT PROFILE")
        print("────────────────────────────")
        for k, v in student_dict.items():
            print(f"{k:45s}: {v}")
        print("────────────────────────────")

    return student_dict


def run_simulation(n=10, verbose=True, show_students=True):
    """
    Runs n random student predictions through the trained model.
    Shows student profiles, predictions, and returns a summary.
    """
    pass_count = 0
    fail_count = 0
    results = []

    for i in range(n):
        print(f"\n===== STUDENT {i+1} =====")
        
        student = generate_random_student(show=show_students)
        pred, prob = predict_student_outcome(**student)

        if pred == 1:
            pass_count += 1
            outcome = "GRADUATE ✅"
        else:
            fail_count += 1
            outcome = "DROPOUT ❌"

        results.append({
            "student_id": i + 1,
            "prediction": outcome,
            "confidence": prob,
            "student_profile": student
        })

        if verbose:
            print(f"\n📌 MODEL DECISION → {outcome}")
            if prob is not None:
                print(f"📊 Confidence → {prob*100:.2f}%")

    summary = {
        "total": n,
        "predicted_graduates": pass_count,
        "predicted_dropouts": fail_count,
        "graduate_rate": pass_count / n,
        "dropout_rate": fail_count / n,
        "results": results
    }

    print("\n📊 FINAL SIMULATION SUMMARY")
    print("────────────────────────────")
    print("Total students        :", n)
    print("Predicted Graduates   :", pass_count)
    print("Predicted Dropouts    :", fail_count)
    print("Graduate Rate         :", round(pass_count / n, 3))
    print("Dropout Rate          :", round(fail_count / n, 3))
    print("────────────────────────────")

    return summary


In [ ]:
sim_results = run_simulation(n=3)


In [ ]:

def test_output_type():
    pred, prob = predict_student_outcome(**generate_random_student())
    assert pred in [0, 1]
    print("✅ test_output_type passed")

def test_probability_bounds():
    pred, prob = predict_student_outcome(**generate_random_student())
    if prob is not None:
        assert 0 <= prob <= 1
    print("✅ test_probability_bounds passed")

def test_extreme_fail_case():
    # Construct a profile with very low curricular performance (strong dropout indicators)
    student = generate_random_student()
    # Set curricular units to 0 (no performance)
    for col in X.columns:
        if 'Curricular units' in col:
            student[col] = 0
    pred, _ = predict_student_outcome(**student)
    # Just verify we get a prediction, not asserting specific outcome
    assert pred in [0, 1]
    print("✅ test_extreme_fail_case passed")

def test_extreme_pass_case():
    # Construct a profile with excellent curricular performance (strong pass indicators)
    student = generate_random_student()
    # Set curricular units to high values
    for col in X.columns:
        if 'approved' in col.lower() or 'grade' in col.lower():
            student[col] = X[col].max()
    pred, _ = predict_student_outcome(**student)
    # Just verify we get a prediction, not asserting specific outcome
    assert pred in [0, 1]
    print("✅ test_extreme_pass_case passed")

def test_feature_alignment():
    student = generate_random_student()
    df_test = pd.DataFrame([student])
    df_test = df_test.reindex(columns=X.columns, fill_value=0)
    assert df_test.shape[1] == X.shape[1]
    print("✅ test_feature_alignment passed")

def test_simulation_return_structure():
    summary = run_simulation(n=5, verbose=False)

    required_keys = {
        "total", "predicted_graduates", 
        "predicted_dropouts", "graduate_rate",
        "dropout_rate", "results"
    }

    assert required_keys.issubset(summary.keys())
    assert summary["total"] == 5
    print("✅ test_simulation_return_structure passed")

def test_simulation_conservation():
    summary = run_simulation(n=8, verbose=False)
    total = summary["predicted_graduates"] + summary["predicted_dropouts"]
    assert total == 8
    print("✅ test_simulation_conservation passed")

def test_multiple_random_predictions_stability():
    preds = []
    for _ in range(20):
        pred, _ = predict_student_outcome(**generate_random_student())
        preds.append(pred)

    # Ensure both classes appear at least once in multiple runs
    assert 0 in preds and 1 in preds
    print("✅ test_multiple_random_predictions_stability passed")

# =========================================================
# ✅✅✅ RUN ALL UNIT TESTS
# =========================================================

def run_all_tests():
    print("\n🧪 RUNNING FULL UNIT TEST SUITE...\n")
    
    print("Testing output type...")
    test_output_type()
    print("Testing probability bounds...")
    test_probability_bounds()
    print("Testing extreme fail case...")
    test_extreme_fail_case()
    print("Testing extreme pass case...")
    test_extreme_pass_case()
    print("Testing feature alignment...")
    test_feature_alignment()
    print("Testing simulation return structure...")
    test_simulation_return_structure()
    print("Testing simulation conservation...")
    test_simulation_conservation()
    print("Testing multiple random predictions stability...")
    test_multiple_random_predictions_stability()

    print("\n🎉 ALL UNIT TESTS PASSED SUCCESSFULLY! ✅")

run_all_tests()
